In [1]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px
from fidlib.basicfunctions import create_ising
from fidlib.variance import VarianceComputer,_abs2,kplus

np.random.seed(1)
%load_ext autoreload
%autoreload 2

In [2]:

qc = EfficientSU2(num_qubits := 6,reps=(reps:=12))
times=(1, 3)
vc = VarianceComputer(    qc=qc,    initial_parameters=None,    times=times,    H=create_ising(num_qubits=num_qubits, j_const=0.5, g_const=-1))


# omegas = np.logspace(-3, -1, 10)
omegas = np.linspace(1e-3,5e-1,20)
variances = []
for o in tqdm(omegas):
    variances.append(vc.direct_compute_variance(400, o))

variances = np.array(variances)

 25%|██▌       | 5/20 [03:41<10:53, 43.60s/it]

In [ ]:
analytical_infidelities=[1-_abs2(np.inner(vc.evolved_states[0], vc.evolved_states[i].conj())) for i in range(times[1])]
display(px.line(x=np.linspace(0,times[0],times[1]),y=analytical_infidelities))
display(px.line(x=omegas,y=[kplus(o)**reps for o in omegas]))

In [ ]:
# Assuming data is your 2D numpy array
data = variances# Replace this with your actual data

# Create steps for the slider, one for each slice along parameter 2
steps = []
for i in range(data.shape[1]):
    step = dict(
        method="restyle",
        args=[{"y": [data[:, i]]}],
        label=str(np.linspace(0,times[0],times[1])[i])
    )
    steps.append(step)

# Create an initial plot for the first slice of data along parameter 2
fig = go.Figure(
    data=[go.Scatter(x=omegas,y=data[:, 0])],
    layout=go.Layout(
        title=f"Interactive Plot num_params={qc.num_parameters}",
        xaxis_title="omega",
        yaxis_title="Variance",
        sliders=[dict(
            active=0,
            pad={"t": 20},
            steps=steps,
        )],
        # yaxis_range=[np.min(data),1],
    )
)

fig.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming data is your 2D numpy array
data = variances

# Create a plot for each entry in data
omegas = times
for i in range(data.shape[1]):
    plt.plot(omegas, data[:, i], label=f'Parameter {i+1}')

# Add title and labels
plt.title("Static Plot")
plt.xlabel("omega")
plt.ylabel("Variance")

# Add legend
plt.legend()

# Show the plot
plt.show()


In [ ]:
# Assuming data is your 2D numpy array
data = variances.T# Replace this with your actual data

# Create steps for the slider, one for each slice along parameter 2
steps = []
for i in range(data.shape[1]):
    step = dict(
        method="restyle",
        args=[{"y": [data[:, i]]}],
        label=str(omegas[i])
    )
    steps.append(step)

# Create an initial plot for the first slice of data along parameter 2
fig = go.Figure(
    data=[go.Scatter(x=np.linspace(0,times[0],times[1]),y=data[:, 0])],
    layout=go.Layout(
        title="Interactive Plot",
        xaxis_title="dt",
        yaxis_title="Variance",
        sliders=[dict(
            active=0,
            pad={"t": 20},
            steps=steps,
        )],
        # yaxis_range=[np.min(data),1],
    )
)

# fig.update_yaxes(type="log")


# Display the interactive plot
fig.show()
